<a href="https://colab.research.google.com/github/eghib22/MLassgn4/blob/main/assgn4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install torch torchvision torchaudio
!pip install wandb
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ekaterineghibradze","key":"012e64a90cccca17cf3aadc7054c6ca4"}'}

In [3]:
!pip install -1 kaggle


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -1


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

 99% 282M/285M [00:00<00:00, 432MB/s]
100% 285M/285M [00:00<00:00, 455MB/s]


In [6]:
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip -d data/

In [7]:
!ls data

example_submission.csv	fer2013.tar.gz	icml_face_data.csv  test.csv  train.csv


In [8]:
!pip install -q wandb
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eghib22 (eghib22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
import pandas as pd
train_df_t = pd.read_csv('data/train.csv')
train_df_t.shape


(28709, 2)

In [10]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(train_df_t, test_size=0.4, random_state=42)

val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [11]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split

class FERDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        """
        Args:
            dataframe: Pandas DataFrame with 'emotion' and 'pixels' columns
            transform: Optional transform to be applied on a sample
        """
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        pixels = self.dataframe.iloc[idx]['pixels']
        image = np.array(pixels.split(), dtype=np.uint8).reshape(48, 48)

        label = self.dataframe.iloc[idx]['emotion']

        image = transforms.ToPILImage()(image)

        if self.transform:
            image = self.transform(image)

        return image, label

In [12]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.1), ratio=(0.3, 3.3)),
])

val_test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = FERDataset(train_df, transform=train_transform)
val_dataset = FERDataset(val_df, transform=val_test_transform)
test_dataset = FERDataset(test_df, transform=val_test_transform)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [13]:
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class FERModel(nn.Module):
    def __init__(self, num_classes=7):
        super(FERModel, self).__init__()

        self.in_channels = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)

        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(ResidualBlock(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [19]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=7, smoothing=0.1):
        super().__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.classes = classes

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.classes - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=-1))

def evaluate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return running_loss / len(data_loader.dataset), 100 * correct / total

def train_model(model, train_loader, val_loader, num_epochs=35, lr=0.001):
    wandb.init(project="fer-pytorch")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    wandb.watch(model, log="all", log_freq=100)

    os.makedirs('models', exist_ok=True)
    model_path = 'models/best_model.pth'
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr,
                                                 steps_per_epoch=len(train_loader),
                                                 epochs=num_epochs)
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = 100 * correct / total

        # Add these print statements
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%')
        print('-' * 50)

        wandb.log({
            "epoch": epoch,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": scheduler.get_last_lr()[0]
        })

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), model_path)
            wandb.save(model_path)
            print(f'New best model saved at {model_path} (Val Acc: {val_acc:.2f}%)')

    print(f'Training complete. Best Validation Accuracy: {best_val_acc:.2f}%')
    return model

def plot_confusion_matrix(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    wandb.log({"conf_matrix": wandb.plot.confusion_matrix(
        y_true=all_labels,
        preds=all_preds,
        class_names=["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]
    )})

if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = FERModel()
    trained_model = train_model(model, train_loader, val_loader)
    test_loss, test_acc = evaluate(trained_model, test_loader, nn.CrossEntropyLoss(), device)
    plot_confusion_matrix(trained_model, test_loader, device)
    wandb.finish()

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
lr,▁▂▃▃▄▅▆▇██████▇▇▇▇▆▆▅▅▄▄▄▃▃▂▂▂▁▁▁▁▁
train_acc,▁▂▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇██████
train_loss,█▇▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
val_acc,▁▂▄▄▃▆▅▄▅▅▅▆▇▆▅▆▇▇▆▇▇▇▇████████████
val_loss,▇█▆▄▇▃▄█▄▅▄▂▂▃▄▃▂▂▂▁▁▁▁▁▂▁▁▂▁▂▂▂▂▂▂
epoch,34
lr,0.0
train_acc,87.80261
train_loss,0.36003
val_acc,65.29084


Epoch [1/35]
Train Loss: 1.7584, Accuracy: 27.84%
Val Loss: 1.7302, Accuracy: 31.28%
--------------------------------------------------
New best model saved at models/best_model.pth (Val Acc: 31.28%)
Epoch [2/35]
Train Loss: 1.5486, Accuracy: 39.85%
Val Loss: 1.8707, Accuracy: 36.57%
--------------------------------------------------
New best model saved at models/best_model.pth (Val Acc: 36.57%)
Epoch [3/35]
Train Loss: 1.3991, Accuracy: 46.21%
Val Loss: 1.4035, Accuracy: 46.24%
--------------------------------------------------
New best model saved at models/best_model.pth (Val Acc: 46.24%)
Epoch [4/35]
Train Loss: 1.3205, Accuracy: 49.56%
Val Loss: 1.5564, Accuracy: 41.87%
--------------------------------------------------
Epoch [5/35]
Train Loss: 1.2725, Accuracy: 51.76%
Val Loss: 1.4154, Accuracy: 45.07%
--------------------------------------------------
Epoch [6/35]
Train Loss: 1.2222, Accuracy: 53.61%
Val Loss: 1.4510, Accuracy: 48.45%
-------------------------------------------

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
lr,▁▂▃▃▄▅▆▇██████▇▇▇▇▆▆▅▅▄▄▄▃▃▂▂▂▁▁▁▁▁
train_acc,▁▂▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇██████
train_loss,█▇▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
val_acc,▁▂▄▃▄▄▃▄▆▆▅▆▅▇▆▆▇▇▇▇▇▇▇▇███████████
val_loss,▆▇▄▅▄▄█▄▂▂▄▂▃▂▂▃▁▂▁▁▂▂▂▁▁▁▁▁▁▂▁▂▂▂▂
epoch,34
lr,0.0
train_acc,87.2627
train_loss,0.37441
val_acc,65.6914
